In [73]:
from scipy.io import arff
import pandas as pd

data = arff.loadarff('cm1.arff')
df = pd.DataFrame(data[0])

df.defects = df.defects.astype('category')
df.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,b'false'
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,b'true'
2,24.0,5.0,1.0,3.0,63.0,309.13,0.11,9.50,32.54,2936.77,...,1.0,0.0,6.0,0.0,15.0,15.0,44.0,19.0,9.0,b'false'
3,20.0,4.0,4.0,2.0,47.0,215.49,0.06,16.00,13.47,3447.89,...,0.0,0.0,3.0,0.0,16.0,8.0,31.0,16.0,7.0,b'false'
4,24.0,6.0,6.0,2.0,72.0,346.13,0.06,17.33,19.97,5999.58,...,0.0,0.0,3.0,0.0,16.0,12.0,46.0,26.0,11.0,b'false'


In [63]:
df.dtypes
df.defects

0      b'false'
1       b'true'
2      b'false'
3      b'false'
4      b'false'
5      b'false'
6      b'false'
7      b'false'
8      b'false'
9      b'false'
10     b'false'
11     b'false'
12     b'false'
13     b'false'
14     b'false'
15     b'false'
16     b'false'
17     b'false'
18     b'false'
19     b'false'
20     b'false'
21     b'false'
22     b'false'
23     b'false'
24     b'false'
25     b'false'
26     b'false'
27     b'false'
28     b'false'
29     b'false'
         ...   
468     b'true'
469     b'true'
470     b'true'
471     b'true'
472     b'true'
473     b'true'
474     b'true'
475     b'true'
476     b'true'
477     b'true'
478     b'true'
479     b'true'
480     b'true'
481     b'true'
482     b'true'
483     b'true'
484     b'true'
485     b'true'
486     b'true'
487     b'true'
488     b'true'
489     b'true'
490     b'true'
491     b'true'
492     b'true'
493     b'true'
494     b'true'
495     b'true'
496     b'true'
497     b'true'
Name: defects, Length: 4

In [160]:
import math
def euclideanDistance(i1, i2):
    sum = 0
    for i in range(len(i1) - 1):
        sum += math.pow((i1[i] - i2[i]), 2)
    return math.sqrt(sum)

euclideanDistance(df.iloc[0].values[:len(df.iloc[0].values)-1], df.iloc[1].values[:len(df.iloc[1].values)-1])
type(df.iloc[0].values[-1])
# bytes('false', 'utf-8')

bytes

In [282]:
def knn(train_df, instance, k):
    distances = []
    numCols = len(train_df.iloc[1])
    for index, row in train_df.iterrows():
        i = row.values
        distances.append([euclideanDistance(instance[:numCols-1], i[:numCols-1]), index, row.defects])
        
    distances.sort(key=lambda x: x[0])
    distances = distances[:k]
    t = 0
    f = 0
    for i in distances:
        if i[2] == bytes('true', 'utf-8'):
            t += 1
        else:
            f += 1
    return t > f

In [366]:
from sklearn.model_selection import KFold

def crossValidation(df, alg, *args):    
    kf = KFold(n_splits=5, shuffle=True)
    kf.get_n_splits(df)
    
    fold = 1
    for train_index, test_index in kf.split(df):
        correct = 0
        if (args):
            radious = args[0](df)
        for i in test_index:
            if args:
                response = alg(df.iloc[train_index], df.iloc[i], 5, radious)
            else:
                response = alg(df.iloc[train_index], df.iloc[i], 5)
            if (response == True and df.iloc[i].defects == bytes('true', 'utf-8') or (response == False and df.iloc[i].defects == bytes('false', 'utf-8'))):
                correct += 1
        print('Fold %d: %f - %d/%d' % (fold, correct/len(test_index), correct, len(test_index)))
        fold += 1

In [367]:
crossValidation(df, knn)

Fold 1: 0.890000 - 89/100
Fold 2: 0.850000 - 85/100
Fold 3: 0.890000 - 89/100
Fold 4: 0.888889 - 88/99
Fold 5: 0.868687 - 86/99


In [277]:
def wknn(train_df, instance, k):
    distances = []
    numCols = len(train_df.iloc[1])
    for index, row in train_df.iterrows():
        i = row.values
        distances.append([euclideanDistance(instance[:numCols-1], i[:numCols-1]), index, row.defects])
    
    distances.sort(key=lambda x: x[0])
    distances = distances[:k]
    t = 0
    f = 0
    for i in distances:
        d = 0
        if i[0] > 0: d = 1/(i[0]**2)
        else: d = 0
        if i[2] == bytes('true', 'utf-8'):
            t += d
        else:
            f += d
    return t > f

In [332]:
crossValidation(df, wknn)

Fold 1: 0.880000 - 88/100
Fold 2: 0.820000 - 82/100
Fold 3: 0.820000 - 82/100
Fold 4: 0.888889 - 88/99
Fold 5: 0.919192 - 91/99


In [343]:
def getRadious(train_df):
    radious = []
    for i, rowI in train_df.iterrows():        
        r = 999999999999999999999999
        ri = 0
        for j, rowJ in train_df.iterrows():
            if rowI.defects != rowJ.defects:
                ri = euclideanDistance(rowI, rowJ) - 1
                if ri < r:
                    r = ri
        radious.append([i, r])
        radiousDf = pd.DataFrame(radious)
        radiousDf = radiousDf.set_index(0)
        radiousDf.columns = ['radious']
    return radiousDf

getRadious(df).head()

,radious
0,
0,1.351595
1,1.351595
2,126.821926
3,122.819903
4,478.375336


In [369]:
def aknn(train_df, instance, k, radious):
    distances = []
    numCols = len(train_df.iloc[1])
    for index, row in train_df.iterrows():
        i = row.values
        r = radious.iloc[index].radious
        distances.append([euclideanDistance(instance[:numCols-1], i[:numCols-1])/r, index, row.defects])
    
    distances.sort(key=lambda x: x[0])
    distances = distances[:k]
    t = 0
    f = 0
    for i in distances:
        d = 0
        if i[0] > 0: d = 1/(i[0]**2)
        else: d = 0
        if i[2] == bytes('true', 'utf-8'):
            t += d
        else:
            f += d
    return t > f

In [370]:
crossValidation(df, aknn, getRadious)

Fold 1: 0.910000 - 91/100
Fold 2: 0.940000 - 94/100
Fold 3: 0.910000 - 91/100
Fold 4: 0.909091 - 90/99
Fold 5: 0.888889 - 88/99
